<a href="https://colab.research.google.com/github/plantehenry/NeuralNetworksFinalProject/blob/main/RawPercentageConvolutionTry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100, CIFAR10
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms.functional import resize
from torchvision.transforms import CenterCrop
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torchsummary import summary
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Define Network

In [ ]:
class CNN(nn.Module):

    def __init__(self, numChannels):
        super(CNN, self).__init__()

        # Convolutional layers:
        self.conv1 = nn.Conv2d(in_channels = numChannels, out_channels = 7, kernel_size = (1, 10), stride = 1)
        self.conv2 = nn.Conv2d(in_channels = 7, out_channels = 15, kernel_size = (1, 5), stride = (1,1))
        self.conv3 = nn.Conv2d(in_channels = 15, out_channels = 15, kernel_size = (1, 3), stride = (1,1))
        # self.conv4 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = (1, 3), stride = (1,1))
        # self.conv5 = nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = (1, 3), stride = (1,1))

        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size = (1,2), stride = (1,2))

        # Batch normalization layers:
        self.batchnorm1 = nn.BatchNorm2d(num_features = 30)
        self.batchnorm2 = nn.BatchNorm2d(num_features = 50)

        # Fully-connected layers:
        self.fc1 = nn.Linear(in_features = 60, out_features= 2)
        self.fc2 = nn.Linear(in_features = 512, out_features= 4)

        self.dropout1 = nn.Dropout2d(p = args.dropoutRate)
        self.dropout2 = nn.Dropout2d(p = args.dropoutRate)
        self.dropout3 = nn.Dropout2d(p = args.dropoutRate)
        self.dropout4 = nn.Dropout(p = args.dropoutRate)

    # Evaluation function
    def evaluate(self, model, dataloader, device):
        model.eval()
        running_loss = 0.0
        criterion = nn.MSELoss()  
        for data in dataloader:
                inputs, true = data
                inputs = inputs.to(device)
                true = true.to(device)
                outputs = model(inputs)

                loss = criterion(outputs, true)

                loss = loss.detach().cpu().numpy()
                running_loss += loss
        return(running_loss/ dataloader.__len__())


    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.maxpool(x)
        # x = self.batchnorm1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.maxpool(x)
        # x = self.batchnorm2(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.dropout3(x)
        # x = self.conv4(x)
        # x = self.relu(x)
        # x = self.conv5(x)
        # x = self.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        # x = self.relu(x)
        # x = self.fc2(x))


        return x

### Data Loaders

In [ ]:
import argparse
from torch.utils.data.dataloader import default_collate
import csv


class TrainData(torch.utils.data.Dataset):
    def __init__(self,args):
        self.args = args
        self.input_sequence , self.output_sequence = self.loadData()
    ## working on giving more than one sequence
    def loadData(self):
        # Read the text
        input_sequence = []
        with open(f"{self.args.workingDir}/{self.args.inputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
              new_row = []
              
              # wrap each value in row in an array
              for val in row:
                new_row.append([float(val)])

              # if this is the first row will need to initialize input sequence with arrays
              while len(new_row) > len(input_sequence):
                input_sequence.append([])
              
              # put each wrapped value in the correct index 
              for idx in range(len(new_row)):
                input_sequence[idx].append(new_row[idx])
        test_len = int(args.test_split * len(input_sequence[0]))
        input_sequence = torch.tensor(input_sequence)[:, 0:test_len, :]



        output_sequence = []
        with open(f"{self.args.workingDir}/{self.args.outputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
            new_row = []
            for val in row:
              new_row.append(float(val) * 100)
            output_sequence.append(new_row)
        output_sequence = torch.tensor(output_sequence)[0:test_len]
        return input_sequence, output_sequence

    def __len__(self):
        # Get the number of sequences for training purpose.
        return len(self.input_sequence[0]) - self.args.seqLength

    # returns tensor is size [num_features, 1, seq_length]
    def __getitem__(self, index):
        return_seq = self.input_sequence[:, index:index+self.args.seqLength, :].transpose(1, 2).detach().clone()

        return (
            return_seq,
            self.output_sequence[index+self.args.seqLength -1],
        )


class ValData(torch.utils.data.Dataset):
    def __init__(self,args):
        self.args = args
        self.input_sequence , self.output_sequence = self.loadData()

    def loadData(self):
        # Read the text
        input_sequence = []
        with open(f"{self.args.workingDir}/{self.args.inputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
              new_row = []
              for val in row:
                new_row.append([float(val)])
              while len(new_row) > len(input_sequence):
                input_sequence.append([])
              for idx in range(len(new_row)):
                input_sequence[idx].append(new_row[idx])
        test_len = int(args.test_split * len(input_sequence[0]))
        input_sequence = torch.tensor(input_sequence)[:, test_len: , :]

        output_sequence = []
        with open(f"{self.args.workingDir}/{self.args.outputFile}.csv", newline='') as csvfile:
          spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
          spamreader.__next__()
          for row in spamreader:
            new_row = []
            for val in row:
              new_row.append(float(val) * 100)
            output_sequence.append(new_row)
        output_sequence = torch.tensor(output_sequence)[test_len: ]
        return input_sequence, output_sequence

    def __len__(self):
        # Get the number of sequences for training purpose.
        return len(self.input_sequence[0]) - self.args.seqLength

    def __getitem__(self, index):
        return_seq = self.input_sequence[:, index:index+self.args.seqLength, :].transpose(1, 2).detach().clone()


        return (
            return_seq,
            self.output_sequence[index+self.args.seqLength - 1],
        )

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('-f')
parser.add_argument('--workingDir', type=str, 
    default="/content/drive/My Drive/Neural Networks Project")
# Can be game_of_thrones or wonder_land:
parser.add_argument('--inputFile', type=str, default="input_data") 
parser.add_argument('--outputFile', type=str, default="output_data") 
parser.add_argument('--maxEpochs', type=int, default=100)
parser.add_argument('--batchSize', type=int, default=12)
parser.add_argument('--seqLength', type=int, default=63)
parser.add_argument('--learningRate', type=float, default=.00001)
parser.add_argument('--dropoutRate', type=float, default=0.3)
parser.add_argument('--test_split', type=float, default=.9)
parser.add_argument('--num_channels', type=float, default=2)
args = parser.parse_args()

In [ ]:
dataset = TrainData(args)
train_loader = DataLoader(dataset,  batch_size=args.batchSize, shuffle=False, drop_last=False, num_workers=2)
valset = ValData(args)
val_loader = DataLoader(valset, batch_size=args.batchSize, shuffle=False, drop_last=False, num_workers=2)
data_iter = iter(train_loader)
print(dataset.__len__())
print(valset.__len__())

4222
414


### The main training and evaluation code starts here

In [ ]:
if __name__ == '__main__':
    # Specify the operation mode:
    # 'train' = training with your train and validation data splits
    # 'eval'  = evaluation of the trained model with your test data split 
    mode = 'train'

    # Path where you plan to save the best model during training
    my_best_model = "/content/drive/MyDrive/Convolution_best_model.pth"

    # Set the device (GPU or CPU, depending on availability)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Currently using device: ", device)

    # Initialize the model and print out its configuration
    model = CNN(numChannels = args.num_channels)
    model.to(device)
    print("\n\nModel summary:\n\n")
    summary(model, input_size=(args.num_channels, 1, 63))

    if mode == "train":

        print("\n\nTraining starts!\n\n")
        
        model.train()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.learningRate)
        # optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
        
        best_val_loss = float('inf')
        for epoch in range(args.maxEpochs):
            running_loss = .0
            print(f"Starting epoch {epoch + 1}")
            bestLoss = float('inf')
            for idx, data in tqdm(enumerate(train_loader), total=len(train_loader)):
                # Get the inputs (data is a list of [inputs, labels])
              
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                loss = loss.detach().cpu().numpy()
                inputs = inputs.detach().cpu().numpy()
                labels = labels.detach().cpu().numpy()
                running_loss += loss
              
            
                currLoss = loss.item()
                
                # Only save models with smallest loss per epoch.
                if currLoss < bestLoss:
                    bestLoss = currLoss
                    # torch.save(model.state_dict(), my_best_model)
            print(f"Epoch ID: {epoch}, 'the ave loss': {running_loss/ train_loader.__len__()}")

            # Evaluate the accuracy after each epoch
            val_loss = model.evaluate(model, val_loader, device)
            print(f"Epoch ID: {epoch}, 'the ave val loss': {val_loss}")
            if val_loss < best_val_loss:
                print(f"Better validation accuracy achieved: {val_loss}")
                best_val_loss = val_loss
                print(f"Saving this model as: {my_best_model}")
                torch.save(model.state_dict(), my_best_model)

    # And here we evaluate the trained model with the test data
    # elif mode == "eval":

    #     print("\n\nValidating the trained model:")
    #     print(f"Loading checkpoint from {my_best_model}")
    #     model.load_state_dict(torch.load(my_best_model))
    #     acc = model.evaluate(model, test_loader, classes, device)
    #     print(f"Accuracy on the test (unknown) data: {acc * 100:.2f}%")

    else:
        print("'mode' argument should either be 'train' or 'eval'")

Currently using device:  cuda:0


Model summary:


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 7, 1, 54]             147
              ReLU-2             [-1, 7, 1, 54]               0
         Dropout2d-3             [-1, 7, 1, 54]               0
         MaxPool2d-4             [-1, 7, 1, 27]               0
            Conv2d-5            [-1, 15, 1, 23]             540
              ReLU-6            [-1, 15, 1, 23]               0
         Dropout2d-7            [-1, 15, 1, 23]               0
         MaxPool2d-8            [-1, 15, 1, 11]               0
            Conv2d-9             [-1, 15, 1, 9]             690
             ReLU-10             [-1, 15, 1, 9]               0
        Dropout2d-11             [-1, 15, 1, 9]               0
        MaxPool2d-12             [-1, 15, 1, 4]               0
           Linear-13                    [-1, 2]     

100%|██████████| 352/352 [00:01<00:00, 200.69it/s]


Epoch ID: 0, 'the ave loss': 3002.8171998804264
Epoch ID: 0, 'the ave val loss': 565.055965750558
Better validation accuracy achieved: 565.055965750558
Saving this model as: /content/drive/MyDrive/Convolution_best_model.pth
Starting epoch 2


100%|██████████| 352/352 [00:01<00:00, 217.67it/s]

Epoch ID: 1, 'the ave loss': 57.02852628447793


...

100%|██████████| 352/352 [00:01<00:00, 220.01it/s]

Epoch ID: 91, 'the ave loss': 11.535006025594406


Epoch ID: 91, 'the ave val loss': 15.878123612063272
Starting epoch 93


 68%|██████▊   | 241/352 [00:01<00:00, 218.52it/s]


KeyboardInterrupt: ignored

In [ ]:
import random
checkpointFile = "/content/drive/MyDrive/Convolution_best_model"

# If using GPU, we need the following line.
model.to(device)

model.load_state_dict(torch.load(f"{checkpointFile}.pth"))


generatingLoader = DataLoader(valset, batch_size=1, shuffle=True)


# Get one data point from the DataLoader
data_point, true = next(iter(generatingLoader))
data_point = data_point.to(device)
print(data_point)
print(true)
model.eval()
torch.no_grad()
print(model(data_point))

tensor([[[[4710.2998, 4642.9902, 4636.4600, 4719.1299, 4652.5000, 4587.8999,
           4594.9600, 4650.3599, 4703.9600, 4733.9902, 4795.4902, 4788.6401,
           4794.2300, 4775.2100, 4778.1401, 4804.5098, 4787.9902, 4693.3901,
           4697.6602, 4655.3398, 4669.1401, 4728.5898, 4733.5601, 4637.9902,
           4632.2402, 4588.0298, 4547.3501, 4471.3799, 4356.3198, 4366.6401,
           4408.4302, 4380.5801, 4336.1899, 4431.7900, 4519.5698, 4566.3901,
           4535.4102, 4482.7900, 4505.7500, 4480.0200, 4547.0000, 4553.2402,
           4506.2700, 4412.6099, 4429.2798, 4455.7500, 4456.0601, 4384.5698,
           4332.7402, 4324.9302, 4155.7700, 4298.3799, 4354.1699, 4363.1401,
           4322.5601, 4401.3101, 4342.1201, 4327.0098, 4202.6602, 4223.1001,
           4252.5498, 4279.5000, 4202.7500]],

         [[ 114.5100,  114.4600,  114.2300,  114.2600,  114.5500,  114.5700,
            114.0700,  114.3300,  114.2900,  114.1500,  114.4300,  113.9900,
            113.9900,  114.15